Código sobre el Análisis de Consumo de Agua en la Ciudad de México 

In [2]:
%pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [36]:
# IMPORTAR PAQUETES NECESARIOS
import pandas as pd  # Para manipulación y análisis de datos tabulares
import requests  # Para realizar solicitudes HTTP a API o sitios web
import geopandas as gpd  # Para trabajar con datos geoespaciales (puntos, líneas, polígonos)
from shapely.geometry import Point, MultiPoint, Polygon
import json  # Para trabajar con datos JSON (formato de intercambio de datos

In [4]:
# Consumo de agua 
agua = 'https://datos.cdmx.gob.mx/dataset/57ffc13a-7207-4b69-a9a9-2fef3fce6331/resource/65a6b1a6-5d6e-49b9-aeed-ca7b22e8de03/download/reportes_agua_hist.csv'
agua = pd.read_csv(agua)
agua.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254730 entries, 0 to 254729
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   folio                    254730 non-null  object 
 1   tipo_de_falla            254725 non-null  object 
 2   quien_atiende            254584 non-null  object 
 3   latitud                  254730 non-null  float64
 4   longitud                 254730 non-null  float64
 5   codigo_postal            238976 non-null  float64
 6   fecha                    254730 non-null  object 
 7   colonia_registro_sacmex  253535 non-null  object 
 8   colonia_datos_abiertos   250610 non-null  object 
 9   alcaldia                 250610 non-null  object 
dtypes: float64(3), object(7)
memory usage: 19.4+ MB


In [5]:
#Diccionario con los nombres de acaldías y su clave de municipio
alcaldia_d = {
    "AZCAPOTZALCO": "002",
    "COYOACAN": "003",
    "CUAJIMALPA DE MORELOS": "004",
    "GUSTAVO A. MADERO": "005",
    "IZTACALCO": "006",
    "IZTAPALAPA": "007",
    "LA MAGDALENA CONTRERAS": "008",
    "MILPA ALTA": "009",
    "ALVARO OBREGON": "010",
    "TLALPAN": "012",
    "XOCHIMILCO": "013",
    "BENITO JUAREZ": "014",
    "CUAUHTEMOC": "015",
    "MIGUEL HIDALGO": "016",
    "VENUSTIANO CARRANZA": "017",
    "TLAHUAC": "011",
}
agua["cve_mun"] = agua["alcaldia"].map(alcaldia_d)

In [6]:
##Crear cartografía con la longitud y latitud

# Crear la columna 'geometry' en el DataFrame original 'agua' con los puntos
agua['geometry'] = gpd.points_from_xy(agua['longitud'], agua['latitud'])

# Convertir 'agua' en un GeoDataFrame
agua = gpd.GeoDataFrame(agua, crs="EPSG:4326")

# Crear una función para generar polígonos a partir de los puntos agrupados por alcaldía
def crear_poligono(grupo):
    puntos = MultiPoint(list(grupo))  # Crear un conjunto de puntos (MultiPoint)
    return puntos.convex_hull  # Generar el polígono más simple (Convex Hull)

# Crear los polígonos por alcaldía
poligonos_por_alcaldia = (
    agua.groupby('alcaldia')['geometry']
    .apply(lambda x: crear_poligono(x))
    .reset_index(name='geometry_poligono')
)

# Añadir los polígonos al DataFrame original
agua = agua.merge(poligonos_por_alcaldia, on='alcaldia', how='left')
agua.info() #Revisar las actualizaciones

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 254730 entries, 0 to 254729
Data columns (total 13 columns):
 #   Column                   Non-Null Count   Dtype   
---  ------                   --------------   -----   
 0   folio                    254730 non-null  object  
 1   tipo_de_falla            254725 non-null  object  
 2   quien_atiende            254584 non-null  object  
 3   latitud                  254730 non-null  float64 
 4   longitud                 254730 non-null  float64 
 5   codigo_postal            238976 non-null  float64 
 6   fecha                    254730 non-null  object  
 7   colonia_registro_sacmex  253535 non-null  object  
 8   colonia_datos_abiertos   250610 non-null  object  
 9   alcaldia                 250610 non-null  object  
 10  cve_mun                  250610 non-null  object  
 11  geometry                 254730 non-null  geometry
 12  geometry_poligono        250610 non-null  geometry
dtypes: float64(3), geometry(2), object(8

In [28]:
print("Ver las fallas",agua["tipo_de_falla"].unique()) 

Ver las fallas ['Falta de agua' 'Mala calidad' 'Fuga' nan]


In [35]:
# Convertir 'fecha' a datetime y extraer el año
agua['fecha'] = pd.to_datetime(agua['fecha'], errors='coerce')
agua['año'] = agua['fecha'].dt.year

# Filtrar para el año 2019 y eliminar filas sin 'alcaldia' ni 'tipo_de_falla'
agua_2019 = agua[(agua['año'] == 2019) & (~agua['tipo_de_falla'].isna()) & (~agua['alcaldia'].isna())]

# Agrupar por tipo de falla y alcaldía, y contar el número de folios
desglose_2019 = agua_2019.groupby(['tipo_de_falla', 'alcaldia']).size().reset_index(name='numero_folios')

# Ordenar los resultados para mejor legibilidad
desglose_2019 = desglose_2019.sort_values(by=['tipo_de_falla', 'numero_folios'], ascending=[True, False])
desglose_2019 


,tipo_de_falla,alcaldia,numero_folios
2,Falta de agua,BENITO JUAREZ,9241
3,Falta de agua,COYOACAN,8426
13,Falta de agua,TLALPAN,7804
6,Falta de agua,GUSTAVO A. MADERO,7549
0,Falta de agua,ALVARO OBREGON,5769
5,Falta de agua,CUAUHTEMOC,4689
8,Falta de agua,IZTAPALAPA,4450
1,Falta de agua,AZCAPOTZALCO,3997
10,Falta de agua,MIGUEL HIDALGO,3867
12,Falta de agua,TLAHUAC,3035
